In [1]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 27.3 MB/s eta 0:00:00


In [2]:
import os
import transformers
import torch
from google.colab import userdata
from datasets import load_dataset
from trl import SFTTrainer
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer


In [24]:
os.environ["HUGGINGFACE_TOKEN"] = userdata.get('HUGGINGFACE_TOKEN')

In [25]:
model_id = "google/gemma-2b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_id, token=os.environ["HUGGINGFACE_TOKEN"] )



model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0},
                                             token=os.environ["HUGGINGFACE_TOKEN"] )


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
text = "Quote: Imagination is more,"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: Imagination is more, than knowledge.

I am a self-taught artist, born in 1985 in


In [29]:
os.environ["WANDB_DISABLED"] = "false"

In [30]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)


In [31]:
from datasets import load_dataset

data = load_dataset("HeshamHaroon/arabic-quotes")
data = data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

In [14]:
data['train']['quote']

['اذا لم يكن لديك هدف ، فاجعل هدفك الاول ايجاد واحد .        ',
 'كل انسان يرى نفسه عالماً بكل شئ هو انسان جاهل ، و كل انسان يريد أن يتعلم بشكل دائم هو الإنسان المعول عليه .        ',
 'من أحب الله رأى كل شئ جميلا .        ',
 'نعمل بالممكن و لا ننسى الطموح .        ',
 'لا تعبدوا الله ليعطي، بل اعبدوه ليرضى، فإن رضي أدهشكم بعطائه.        ',
 'عندما يصبح البحر ساكناً ، يصبح الجميع بحارة ماهرين .        ',
 'اهتم بأن تحصل على ما تحبه، و الا ستكون مجبراً على ان تقبل ما تحصل عليه .        ',
 'لا تكن ممن يلعن ابليس علانية و يطيعه سرا .        ',
 'ما دمت مؤمناً بهدفك و متوكلا على الله ، فانك ستتحول الى مغناطيس جبار ، يجتذب اليه كل ما هو ضروري من الاشياء و الاحداث و الاشخاص و المعلومات لتحقيق هذا الهـدف .        ',
 'أصعب معركة في حياتك عندما يدفعك الناس إلي ان تكون شخصاً آخر.        ',
 'لسان الناس كتاب على الارض ، فلا تهمل قراءته ، و لا تصدق كل ما تقرأه فيه .        ',
 'سر النجاح في الحياة أن تواجه مصاعبها بثبات الطير في ثورة العاصفة .        ',
 '"يوماً مـا" ، ليس من أيام الأسبوع .    

In [32]:
def formatting_func(example):
    text = f"Quote: {example['quote'][0]}\nAuthor: {example['author'][0]}"
    return [text]

In [33]:
data['train']


Dataset({
    features: ['quote', 'author', 'tags', 'input_ids', 'attention_mask'],
    num_rows: 3778
})

In [34]:
trainer = SFTTrainer(
    model=model,
    train_dataset=data["train"],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=2,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    formatting_func=formatting_func,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:223: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [35]:
trainer.train()


Step,Training Loss
1,4.632700
2,4.632700
3,4.526500
4,4.277800
5,4.001500
6,3.724700
7,3.451400
8,3.182200
9,2.915700
10,2.644500


TrainOutput(global_step=200, training_loss=0.3073514815419912, metrics={'train_runtime': 255.4207, 'train_samples_per_second': 3.132, 'train_steps_per_second': 0.783, 'total_flos': 351334158336000.0, 'train_loss': 0.3073514815419912, 'epoch': 200.0})

In [38]:
text = "Quote: الأم ... شمعة مقدسة"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: الأم ... شمعة مقدسة تحملها في طياتها من أمي.        
Author: توماس اديسون        


In [37]:
text = "Quote: ،اذا لم يكن لديك هدف"
device = "cuda:0"
inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Quote: ،اذا لم يكن لديك هدف ، فاجعل هدفك الاول ايجاد واحد .        
Author: وليام شكسب
